# ZHCPA Dingtalk

## Retrieve access token

In [162]:
import json
from pprint import pprint
from typing import List, Optional, Tuple
from alibabacloud_dingtalk.oauth2_1_0.client import Client as DingtalkClient
from alibabacloud_tea_openapi import models as open_api_models
from alibabacloud_dingtalk.oauth2_1_0 import models as dingtalk_oauth_models
import os
from dotenv import load_dotenv
from requests import request
from typing import Any, Dict, Callable
from functools import partial

load_dotenv()

app_key = os.getenv("APP_KEY")
app_secret = os.getenv("APP_SECRET")
admin_opuserid = os.getenv("ADMIN_OPUSERID")

if not app_key or not app_secret or not admin_opuserid:
    raise Exception("APP_KEY and APP_SECRET must be set")

config = open_api_models.Config()
config.protocol = "https"
config.region_id = "central"
dingtalk_client = DingtalkClient(config)

get_access_token_request = dingtalk_oauth_models.GetAccessTokenRequest(
    app_key=app_key, app_secret=app_secret
)
try:
    response = dingtalk_client.get_access_token(get_access_token_request)

    if getattr(response, "status_code") != 200:
        raise Exception("Request not ok")

    access_token = getattr(response.body, "access_token")
    if not access_token:
        raise Exception("No access token presented in response body")
except Exception as err:
    raise err


In [163]:
def api(endpoint: str, data: Dict[str, Any], method: str = "POST"):
    try:
        response = request(
            method,
            endpoint,
            params={"access_token": access_token},
            data=data,
        )
        if not response.ok:
            raise Exception("Request not ok")
        
        if not response.json()["success"]:
            raise Exception(f"Dingtalk request not ok: {response.json()['errmsg']}")
        
        return response.json()["result"]

    except Exception as err:
        raise err

In [164]:
def generate_depagination_logic(fetch_from_server: Callable[[int], Tuple[List[Any], Optional[int]]]):
    def wrapper():
        all_data: List[Any] = []
        offset: Optional[int] = 0
        while offset is not None:
            partial_data, offset = fetch_from_server(offset)
            all_data.extend(partial_data)

        return all_data
    
    return wrapper

In [165]:
from alibabacloud_tea_util.models import RuntimeOptions

runtime_options = RuntimeOptions()

## Get all operation users IDs

In [166]:
def get_opuserid_list(
    offset_and_size: Optional[Tuple[int, int]] = None, status_list: str = "2,3,5,-1"
) -> List[str]:
    def fetch_from_server(offset: int, size: int) -> Tuple[List[str], Optional[int]]:
        try:
            data = api(
                "https://oapi.dingtalk.com/topapi/smartwork/hrm/employee/queryonjob",
                {
                    "status_list": status_list,
                    "offset": offset,
                    "size": size,
                },
            )
            return data["data_list"], data.get("next_cursor")

        except Exception as err:
            raise err

    if offset_and_size:
        return fetch_from_server(*offset_and_size)[0]

    return generate_depagination_logic(partial(fetch_from_server, size=50))()

## Get vacation types

In [167]:
from alibabacloud_dingtalk.attendance_1_0 import models as dingtalk_attendance_models

def get_vacation_types() -> List[dingtalk_attendance_models.AddLeaveTypeResponseBodyResult]:
    try:
        data = api(
            "https://oapi.dingtalk.com/topapi/attendance/vacation/type/list",
            {
                "op_userid": admin_opuserid,
                "vacation_source": "all"
            },
        )
        return data

    except Exception as err:
        raise err

## Get operation users leave records

In [168]:
from alibabacloud_dingtalk.attendance_1_0 import models as dingtalk_attendance_models
from alibabacloud_dingtalk.attendance_1_0.client import (
    Client as DingtalkAttendanceClient,
)


def get_opusers_leave_records(
    leave_code: str, opuserids: List[str]
) -> List[dingtalk_attendance_models.GetLeaveRecordsResponseBodyResultLeaveRecords]:
    def fetch_from_server(
        pageOffset: int,
    ) -> Tuple[
        List[dingtalk_attendance_models.GetLeaveRecordsResponseBodyResultLeaveRecords],
        Optional[int],
    ]:
        req = dingtalk_attendance_models.GetLeaveRecordsRequest(
            op_user_id=admin_opuserid,
            user_ids=opuserids,
            leave_code=leave_code,
            page_number=pageOffset,
            page_size=200,
        )
        headers = dingtalk_attendance_models.GetLeaveRecordsHeaders(
            x_acs_dingtalk_access_token=access_token
        )

        try:
            response = DingtalkAttendanceClient(config).get_leave_records_with_options(
                req, headers, runtime=runtime_options
            )
            if getattr(response, "status_code") != 200:
                raise Exception("Request not ok")
            
            if not getattr(response.body, "success"):
                raise Exception(f"Dingtalk request not ok")
            
            data = response.body.result

            return (
                data.leave_records,
                pageOffset + 1 if getattr(data, "has_more") else None,
            )
        except Exception as err:
            raise err

    return generate_depagination_logic(fetch_from_server)()

## Lab

In [169]:
import pandas as pd

In [170]:
all_opuserids = get_opuserid_list()

In [171]:
opuserids_df = pd.DataFrame(all_opuserids)
opuserids_df

,0
0,011363164867487807455
1,012314351753-368250697
2,2707060727-1037595066
3,012540471117-1641702324
4,012540442450-1155240308
...,...
288,510554114782879
289,022420693151-1319269698
290,235629401145-1012136231
291,235629402347-573287760


In [172]:
vacation_types = get_vacation_types()

In [173]:
vacation_types_df = pd.DataFrame(vacation_types)
vacation_types_df

,freedom_leave,hours_in_per_day,leave_code,leave_hour_ceil,leave_name,leave_time_ceil_min_unit,leave_view_unit,natural_day_leave,paid_leave,source,when_can_leave
0,True,800,f60377e7-9c60-48bb-91e2-5d02f4415361,up,年假(小時),halfHour,hour,false,False,inner,entry
1,True,800,c4d0316f-c234-4d89-8c66-c90b5931d072,,年假(天),NaN,day,false,False,inner,entry
2,True,800,c02aa600-8dce-4b63-9945-35d346464ef6,down,病假,halfHour,hour,false,False,inner,entry
3,True,800,a9047b4b-0d7f-4d7c-a57d-6cb89f7d3b91,,考試假,hour,day,false,False,inner,entry
4,True,800,52ed3423-0612-41c4-ace8-dc6b0c098030,down,侍/產假,hour,hour,false,True,inner,entry
5,True,800,385e7625-a0fb-4337-8226-43a2f3eac4a2,,恩恤假,hour,day,false,False,inner,entry


In [174]:
df = vacation_types_df.loc[vacation_types_df["leave_name"] == "年假(天)", "leave_code"]
leave_code = df.iloc[0]
opuserids: List[str] = opuserids_df.iloc[:10, 0].tolist()
opusers_leave_records = get_opusers_leave_records(leave_code, opuserids)

In [175]:
opusers_leave_records_df = pd.DataFrame(map(lambda record: record.__dict__, opusers_leave_records))
opusers_leave_records_df

,cal_type,end_time,gmt_create,gmt_modified,leave_code,leave_reason,leave_record_type,leave_status,leave_view_unit,op_user_id,quota_id,record_id,record_num_per_day,record_num_per_hour,start_time,user_id
0,None,1688745599000,1688639997000,1688692898000,c4d0316f-c234-4d89-8c66-c90b5931d072,,leave,success,day,None,None,9c74ef8c-0bb9-47c3-9d18-0294d72774c1,100,None,1688659200000,0127504627051249095401
1,None,1687795199000,1685928357000,1685930230000,c4d0316f-c234-4d89-8c66-c90b5931d072,,leave,success,day,None,None,324a0a62-d4a5-4d66-a2f2-ec1d274aff6a,1500,None,1685894400000,0127504627051249095401
2,None,1677859199000,1677803725000,1677805195000,c4d0316f-c234-4d89-8c66-c90b5931d072,,leave,abort,day,None,None,f65d92c8-ea1f-4525-9975-20d9c259f877,100,None,1677772800000,0127504627051249095401
3,None,1680537599000,1677114059000,1677123786000,c4d0316f-c234-4d89-8c66-c90b5931d072,,leave,success,day,None,None,0cf92e80-87f8-43db-810a-21184f5df954,100,None,1680451200000,012808442227633917809
4,None,1680796799000,1676952874000,1676953682000,c4d0316f-c234-4d89-8c66-c90b5931d072,,leave,success,day,None,None,1a92b368-ed74-4b58-a634-cb2d5ab756e6,200,None,1680537600000,012808442227633917809
5,None,1674835199000,1674199144000,1674200150000,c4d0316f-c234-4d89-8c66-c90b5931d072,,leave,success,day,None,None,bc23dfc8-9c2a-49db-b516-8a669ae9dda9,200,None,1674662400000,0127504627051249095401
